# 要約 
このJupyterノートブックは、Kaggleの「LMSYS - Chatbot Arena」における、LLM（大規模言語モデル）34Bを用いた問題解決に取り組んでいます。具体的には、モデルの推論を5時間で完了させる方法を示しています。以下に、ノートブックの主な内容と使用手法について要約します。

### 取り組んでいる問題
- **迅速なモデル提出**: 大規模言語モデルを短時間で提出する方法を求めています。具体的には、LLM 34Bモデルを用いて、提出の際に求められるレスポンスを迅速に生成することを目的としています。

### 使用手法・ライブラリ
1. **vLLM**: 高速なLLM推論ライブラリであり、推論速度を向上させるために使用されています。環境によってはエラーが出るため、再インストールが推奨されています。

2. **AWQ（Adaptive Weight Quantization）**: 4ビット量子化を用いることで、GPUのVRAMを効率的に使用する手法です。これにより、大きなモデルをメモリ制約のある環境で実行可能にしています。

3. **トークン制限**: 
   - **入力サイズ**: 最大1024トークンに制限し、速度を向上させます。
   - **出力サイズ**: 生成される応答のトークンを1トークンに制限することで、処理速度を加速させています。

4. **トークナイザー**: モデルに最適化されたトークナイザーを使用し、入力を整形します。

5. **プロンプト工夫**: モデルが出力するトークンを制御し、特定のトークン(A、B、tie)に対して予測を強化するためのロジットプロセッサーを定義しています。これにより、ユーザーが選択するべき応答を促進します。

6. **CSVファイルの生成**: 提出用のCSVファイルを作成するプロセスが含まれており、推論結果を適切な形式で出力します。

7. **CVスコア計算**: 提出を通じて、交差検証スコア（CVスコア）を計算し、結果の性能を評価します。

このノートブックは、モデルのトレーニングや微調整に加え、推論速度や精度を管理するための細やかな配慮がなされており、特にリソースの限られた環境での実用的なアプローチを提示しています。

---


# 用語概説 
以下は、初心者がつまずきそうな専門用語の簡単な解説です。特に、実務経験が少ない場合や、この特定のノートブックに関連した知識が必要な用語に焦点を当てています。

1. **vLLM**:
   - *Very Large Language Model*の略。大規模な言語モデルの推論のために設計された非常に高速なライブラリ。このライブラリを使用することで、サンプリングやトークン生成が効率的に行えます。

2. **AWQ (Adaptive Weight Quantization)**:
   - 重みを量子化する手法。モデルのストレージや推論時のメモリ消費を削減するために、モデルの重みをより少ないビットで表現します。この手法は特に4ビット量子化で使用され、性能とメモリ利用のバランスを取るのに役立ちます。

3. **トークン**:
   - モデルが処理する基本的な単位で、通常は単語や文字の断片。入力テキストをトークンに分割し、これを基にしてモデルが予測を行います。

4. **ロジットプロセッサ (LogitsProcessor)**:
   - 出力のスコア（ロジット）を操作するためのクラス。このクラスを通じて、モデルの出力トークンの選擇を制御したり、特定のトークンを優先させたりできます。

5. **グラデーションチェックポイント (Gradient Checkpointing)**:
   - メモリを節約するための手法で、バックプロパゲーション時に全ての中間結果を保存せず、いくつかの重要な中間結果だけを保存します。この手法により、より大きなモデルの訓練が可能になりますが、計算コストが増加します。

6. **信頼されたリモートコード (trust_remote_code)**:
   - リモートのコードを実行する際の設定。これを`True`に設定すると、リモートのソースから取得したコードを実行することを許可します。この設定は、セキュリティ上のリスクを伴うことがあります。

7. **半精度 (half precision)**:
   - 浮動小数点数の表現方法の一つで、32ビット（single precision）よりも少ない16ビットで数値を表現します。これにより、計算速度が向上し、メモリ利用量も減少するため、特に深層学習でよく使用されます。

8. **トップP (top-p sampling)**:
   - 確率的なトークンサンプリング手法の一つで、選択肢を累積確率に基づいて制限します。例えば、累積確率が95%になるまでのトークン群を考慮し、その中からランダムにサンプリングすることで、多様性を持った出力が得られます。

9. **温度 (temperature)**:
   - サンプリングの際に、出力の多様性を調整するパラメータ。低い温度は決定的な出力をもたらし、高い温度はより多様でランダムな出力を生成します。

10. **CVスコア (Cross-Validation Score)**:
   - モデルの一般化性能を評価するために、異なるデータセットで訓練と評価を繰り返した際のスコア。通常は、様々なトレーニングデータの分割を使用して、モデルがデータにどれだけ適応できるかを計測します。

これらの解説が、ノートブックの内容を理解する上で役立つことを願っています。

---


# LLM 34Bモデルを5時間で提出する方法！
このノートブックでは、わずか5時間でLLM 34Bモデルを提出する方法を示します！すごいですね！主なポイントは以下の通りです：
* 速度のためにvLLMを使用する
* GPUのVRAM不足を避けるためにAWQ 4ビット量子化を使用する
* 入力サイズを1024トークンに制限する（速度向上のため）
* 出力サイズを1トークンに制限する（速度向上のため）

# vLLMのインストール
vLLMパッケージは非常に高速なLLM推論ライブラリです！KaggleノートブックにインストールされているvLLMではエラーが発生するため、再インストールが必要です。以下のコードはノートブック[こちら][1]から取得したものです。

[1]: https://www.kaggle.com/code/lewtun/numina-1st-place-solution

In [ ]:
import os, math, numpy as np  # osモジュール、mathモジュール、numpyライブラリをインポートします。

# 環境変数CUDA_VISIBLE_DEVICESを設定します。
# これにより、どのGPUデバイスを使用するかを指定できます。
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"  # 使用するGPUデバイスとして0番と1番を指定します。

In [ ]:
%%time  # このセルの実行にかかる時間を計測します。

# torchパッケージをアンインストールします。
# -yオプションを使用して、確認なしでアンインストールを行います。
!pip uninstall -y torch  

# vllmパッケージをアップグレードしてインストールします。
# --no-indexオプションを使用してPyPIからのインストールを無効にし、
# --find-linksオプションで指定したディレクトリからパッケージを探します。
!pip install -U --no-index --find-links=/kaggle/input/vllm-whl -U vllm  

# grpcioの特定バージョンをアップグレードしてインストールします。
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/grpcio-1.62.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl  

# rayの特定バージョンをアップグレードしてインストールします。
!pip install -U --upgrade /kaggle/input/vllm-t4-fix/ray-2.11.0-cp310-cp310-manylinux2014_x86_64.whl

# vLLMを使って34B量子化モデルをロードする！
LLM 34B Bagelモデルを[こちら][1]からロードして使用します。これは非常に強力なモデルです。

[1]: https://huggingface.co/jondurbin/bagel-34b-v0.2

In [ ]:
import vllm  # vLLMライブラリをインポートします。

# LLMオブジェクトを作成します。
llm = vllm.LLM(
    "/kaggle/input/bagel-v3-343",  # モデルのパスを指定します。
    quantization="awq",  # AWQ量子化を使用します。
    tensor_parallel_size=2,  # テンソル並列処理のサイズを2に設定します。
    gpu_memory_utilization=0.95,  # GPUメモリの使用率を95%に設定します。
    trust_remote_code=True,  # リモートコードを信頼する設定です。
    dtype="half",  # データ型を半精度に設定します（メモリ効率を向上します）。
    enforce_eager=True,  # イージー実行を強制します。
    max_model_len=1024,  # モデルの最大長を1024トークンに設定します。
    #distributed_executor_backend="ray",  #（コメントアウト）分散実行のバックエンドをRayに設定します。
)

# トークナイザーを取得します。
tokenizer = llm.get_tokenizer()  # モデルに関連付けられたトークナイザーを取得します。

# テストデータをロードする
**コミット**の際にはCVスコアを計算するために128行のトレーニングデータをロードします。**提出**の際にはテストデータをロードします。

In [ ]:
import pandas as pd  # pandasライブラリをインポートします。

VALIDATE = 128  # バリデーション用の行数を128に設定します。

# テストデータをCSVファイルから読み込みます。
test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/test.csv") 

# 読み込んだテストデータの行数が3の場合、
# トレーニングデータを読み込み、最初の128行を取得します。
if len(test) == 3:
    test = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")
    test = test.iloc[:VALIDATE]  # 最初の128行を取得します。

# テストデータの形状を出力します。
print(test.shape)  # テストデータの行数と列数を表示します。

# テストデータの最初の1行を表示します。
test.head(1)  # テストデータの最初の1行を表示します。

# プロンプトの工夫
ゼロショットLLMを提出したい場合、CVスコアを改善するためにさまざまなシステムプロンプトを試す必要があります。モデルをファインチューニングする場合、システムプロンプトはそれほど重要ではなくなります。なぜなら、モデルはターゲットから何をすべきかを学ぶため、どのシステムプロンプトを使用しても影響を受けないからです。

ロジットプロセッサを使用して、モデルが私たちが興味を持っている3つのトークンを出力するよう強制します。

In [ ]:
from typing import Any, Dict, List  # 必要な型をインポートします。
from transformers import LogitsProcessor  # LogitsProcessorクラスをインポートします。
import torch  # PyTorchライブラリをインポートします。

choices = ["A", "B", "tie"]  # 選択肢を定義します。

KEEP = []  # 出力トークンとして保持するトークンIDのリストを初期化します。
for x in choices:
    c = tokenizer.encode(x, add_special_tokens=False)[0]  # トークンをエンコードし、リストから最初の要素を取得します。
    KEEP.append(c)  # 取得したトークンIDをKEEPリストに追加します。
print(f"Force predictions to be tokens {KEEP} which are {choices}.")  # どのトークンを強制するか出力します。

class DigitLogitsProcessor(LogitsProcessor):
    def __init__(self, tokenizer):
        self.allowed_ids = KEEP  # 許可されたトークンIDを初期化します。
        
    def __call__(self, input_ids: List[int], scores: torch.Tensor) -> torch.Tensor:
        scores[self.allowed_ids] += 100  # 許可されたトークンIDのスコアを100増加させます。
        return scores  # 修正されたスコアを返します。

In [ ]:
sys_prompt = """以下のプロンプトと2つの応答を読み、それぞれの応答がどちらが優れているかを判断してください。
もし応答が比較的新しい場合は、「tie」と返信してください。それ以外の場合は、「A」または「B」と応答し、どちらが優れているかを示してください。"""

In [ ]:
SS = "#"*25 + "\n"  # "#"を25個連結し、改行を追加した文字列をSSに設定します。

In [ ]:
all_prompts = []  # プロンプトを保存するリストを初期化します。
for index, row in test.iterrows():  # テストデータの各行をイテレートします。
    
    # プロンプト、応答A、応答Bを取得し、nullを空文字に置き換えます。
    a = " ".join(eval(row.prompt, {"null": ""}))  # プロンプトを評価し、nullを空文字に置き換えた後、スペースで結合します。
    b = " ".join(eval(row.response_a, {"null": ""}))  # 応答Aを評価・結合します。
    c = " ".join(eval(row.response_b, {"null": ""}))  # 応答Bを評価・結合します。
    
    # フォーマットされたプロンプトを作成します。
    prompt = f"{SS}PROMPT: " + a + f"\n\n{SS}RESPONSE A: " + b + f"\n\n{SS}RESPONSE B: " + c + "\n\n"
    
    # システムプロンプトとフォーマットされたプロンプトを結合します。
    formatted_sample = sys_prompt + "\n\n" + prompt
    
    # フォーマットされたサンプルをリストに追加します。
    all_prompts.append(formatted_sample)  # フォーマットされたサンプルをall_promptsリストに追加します。

# テストを推論する
高速なvLLMを使用してテストを推論します。vLLMに対し、最初のトークンで予測される上位5つのトークンの確率を出力するように依頼します。また、推論速度を向上させるために、予測を1トークンに制限します。

128のトレーニングサンプルを推論するのにかかる時間に基づいて、25,000のテストサンプルを推論するのにかかる時間を推測できます。

In [ ]:
%%time  # このセルの実行にかかる時間を計測します。

from time import time  # timeモジュールからtime関数をインポートします。
start = time()  # 実行開始時刻を記録します。

# ロジットプロセッサを定義します。
logits_processors = [DigitLogitsProcessor(tokenizer)]
# vLLMを使用して応答を生成します。
responses = llm.generate(
    all_prompts,  # すべてのプロンプトを指定します。
    vllm.SamplingParams(
        n=1,  # 各プロンプトに対して返す出力シーケンスの数。
        top_p=0.9,  # 上位トークンを考慮する際の累積確率を制御する浮動小数点数。
        temperature=0,  # サンプリングのランダム性。
        seed=777,  # 再現性のためのシード。
        skip_special_tokens=True,  # 出力で特殊トークンをスキップするかどうか。
        max_tokens=1,  # 各出力シーケンスに生成する最大トークン数。
        logits_processors=logits_processors,  # 使用するロジットプロセッサ。
        logprobs=5  # 上位5つのトークン確率を出力。
    ),
    use_tqdm=True  # プログレスバーを表示するためのオプション。
)

end = time()  # 実行終了時刻を記録します。
elapsed = (end - start) / 60.  # 経過時間を分単位で計算します。
print(f"{VALIDATE}サンプルの推論に{elapsed}分かかりました！")  # 推論にかかった時間を出力します。

In [ ]:
submit = 25_000 / 128 * elapsed / 60  # 25,000サンプルの推論にかかる時間を計算します。
print(f"提出には{submit}時間かかります")  # 推論にかかる時間を出力します。

# 推論確率の抽出
これからvLLMの予測から「A」、「B」、「tie」の確率を抽出します。

In [ ]:
results = []  # 推論結果を格納するリストを初期化します。
errors = 0  # エラーのカウンタを初期化します。

for i, response in enumerate(responses):  # レスポンスの各要素をイテレートします。
    try:
        x = response.outputs[0].logprobs[0]  # 最初の出力のロジットを取得します。
        logprobs = []  # ログ確率を格納するリストを初期化します。
        for k in KEEP:  # KEEPに含まれる各トークンIDについて
            if k in x:  # トークンIDがログ確率に存在する場合
                logprobs.append(math.exp(x[k].logprob))  # ログ確率の指数を計算して追加します。
            else:
                logprobs.append(0)  # トークンIDが存在しない場合は0を追加します。
                print(f"bad logits {i}")  # 不正なロジットが見つかったことを記録します。
        logprobs = np.array(logprobs)  # ログ確率リストをNumPy配列に変換します。
        logprobs /= logprobs.sum()  # 確率を正規化します。
        results.append(logprobs)  # 結果リストに追加します。
    except:  # エラーが発生した場合
        # print(f"error {i}")  # エラーを記録（コメントアウトされています）。
        results.append(np.array([1/3., 1/3., 1/3.]))  # 一様分布の確率を追加します。
        errors += 1  # エラーカウンタを増やします。
        
print(f"{i+1}回の推論のうちエラーは{errors}回発生しました。")  # エラーの総数を出力します。
results = np.vstack(results)  # 結果を垂直スタックして2次元配列を作成します。

# 提出用CSVを作成する

In [ ]:
sub = pd.read_csv("/kaggle/input/lmsys-chatbot-arena/sample_submission.csv")  # サンプル提出CSVファイルを読み込みます。

# テストデータの長さがVALIDATEでない場合、結果をサブミッション用データフレームに代入します。
if len(test) != VALIDATE:
    sub[["winner_model_a", "winner_model_b", "winner_tie"]] = results  # 結果を対応する列に割り当てます。
    
sub.to_csv("submission.csv", index=False)  # 提出用CSVを作成します。インデックスは出力しません。
sub.head()  # サブミッションデータフレームの最初の数行を表示します。

# CVスコアを計算する

In [ ]:
if len(test) == VALIDATE:  # テストデータの長さがVALIDATEと等しい場合
    true = test[['winner_model_a', 'winner_model_b', 'winner_tie']].values  # 真のラベルを取得します。
    print(true.shape)  # 真のラベルの形状を出力します。

In [ ]:
if len(test) == VALIDATE:  # テストデータの長さがVALIDATEと等しい場合
    from sklearn.metrics import log_loss  # log_loss関数をインポートします。
    print(f"CV loglossは {log_loss(true, results)} です")  # CVのログ損失を計算して出力します。

# コメント

> ## Cody_Null
> 
> すごい仕事ですね、クリス。vLLMのパフォーマンス低下について何か知っていますか？私は何も知らないので、他のノートブックでこれを使う際の期待値を知りたいと思っています。
> 
> 
> > ## Chris Deotte（トピック作成者）
> > 
> > vLLMによるパフォーマンス低下はないと思います。vLLMパッケージは単に私たちのモデルを実行するだけです。選択するオプション（vllm.SamplingParams、量子化の有無、量子化の種類、max_model_length、トークナイザの切り捨てなど）がパフォーマンスに影響を与えます。
> > 
> > 
> > 
> > > ## Cody_Null
> > > 
> > > すばらしい！ONNXに似ているのか、正確に私が必要としているものなのかわからなかったです（笑）。
> > > 
> > > 
> > > 
> > > ## Chris Deotte（トピック作成者）
> > > 
> > > モデルの変換（ONNXのような）は必要ありません。Hugging Faceから非量子化モデルと量子化モデルの両方を直接取得し、推論を実行できます。（つまり、モデルは全く変更されません）。
> > > 
> > > 

---

> ## SeshuRaju 🧘‍♂️
> 
> [@cdeotte](https://www.kaggle.com/cdeotte) ありがとうございます。
> 
> このモデルはSFTまたはDPOでファインチューニングされていますか？
> 
> ファインチューニングにはどれほどのGPUが必要ですか？
> 
> 
> > ## Chris Deotte（トピック作成者）
> > 
> > このモデルはSFTでファインチューニングされています。選択したLoRAランクパラメータr、max_model_len、バッチサイズ、およびトレーニングデータの量に応じて、任意のGPU数でファインチューニングできます。
> > 
> > ファインチューニング中に、34Bを4ビットに量子化し、サイズを20GBに削減します。したがって、要件は、GPUの総VRAMがトレーニングに十分な20GBを超えることです。Kaggleの2xT4（合計VRAM 32GB）上で、上記のパラメータを減らせばファインチューニングできると思います。
> > 
> > 
> > 
> > > ## SeshuRaju 🧘‍♂️
> > > 
> > > 限られたKaggle GPU時間のため、16GB VRAMで訓練しようとしています。バッチ=1、r=3、max_model_len=1024で訓練する他の方法を探しています。
> > > 
> > > 
> > > > ## Chris Deotte（トピック作成者）
> > > > 
> > > > 16GB VRAMで34B LLMをファインチューニングする方法はわかりません。最小要件は32GB VRAMに近いと思います。効率的なファインチューニングに関する詳細を説明した素敵なYouTubeビデオがあります [こちら](https://www.youtube.com/watch?v=XpoKB3usmKc) をご覧ください。ファインチューニング中には、次のメモリが必要です：
> > > > 
> > > > - モデルの重み
> > > > 
> > > > - 勾配
> > > > 
> > > > - オプティマイザ
> > > > 
> > > > メモリを最も削減するには、4ビット量子化されたモデルをロード（QLoRAを使用してモデル重みのメモリ使用量を削減）し、llm_int8_enable_fp32_cpu_offloadを使用してGPUとCPUメモリ間でモデルの重みをスワップできるようにし、PEFTを使用して少ないrパラメータ（QLoRA）でファインチューニングし、勾配チェックポイントを使用して（勾配メモリ使用量を削減）、paged_adam_8bitオプティマイザを使用します（小さなバッチと小さな最大トークン長で）。このオプティマイザは、必要に応じてGPUからCPUに保存された変数をスワップし、オプティマイザ変数のサイズを削減するために8ビットを使用します。
> > > 
> > > 

---

> ## ano
> 
> この素晴らしいモデルを共有していただきありがとうございます！トレーニングに使用したデータセットについて教えていただけますか？トレーニングデータセット全体を使用しましたか、それともその一部ですか？または外部データセットを使用しましたか？
> 
> 
> > ## Chris Deotte（トピック作成者）
> > 
> > このモデルはコンペティションデータの80%でファインチューニングしました。5行ごとにすべてのデータを除外しました。したがって、正しい検証は以下のデータを使用することです pd.read_csv("train.csv").iloc[0::5]。現在、このノートブックでは pd.read_csv("train.csv").iloc[:128] を使用していますが、より正確な迅速な検証は 
> > 
> > ```
> > VALIDATE = 128
> > test = test.iloc[0:VALIDATE*5:5]
> > 
> > ```
> > 
> > その後、最初のインデックス % 5 == 0 のサンプル128を使用します。
> > 
> > 

---

> ## JM
> 
> 提出段階で実際に5時間かかっていますか？私のはずっとオーバーしています。
> 
> > ## Chris Deotte（トピック作成者）
> > 
> > いいえ、私は5時間かかっていません。私のは8から9時間かかっています。このノートブックの別のバージョンをリリースするときに、コードセル#10の時間推定コードを修正します（「推論エラー」を削除することが含まれます）。また、導入文を「5時間」ではなく「9時間未満」と更新します。
> > 
> > 
> > 
> > > ## ano
> > > 
> > > 推論時間の推定（5時間）が実際のもの（8〜9時間）と異なる理由に心当たりはありますか？私は1000サンプルの推論を試してみましたが、25000サンプルのための推定時間（5-6時間）は似たようなものでした。
> > > 
> > > 

---

> ## Luan Ngo Dinh
> 
> こんにちは、GPTQの量子化結果がAWQとどう比較されるか、またKaggleでの可行性についてお尋ねしますか？
> 
> > ## Chris Deotte（トピック作成者）
> > 
> > GPTQを使用するためには、vLLMパラメータで量子化="gptq"を設定し、事前にモデルをGPTQとして保存します。私はKaggleで試したことはありませんが、過去にT4 GPUでAWQとGPTQをどちらも成功裏に利用してきました。それぞれの精度は基本的に似ています。また、推論時間も似ています。
> > 
> > 
> > 
> > > ## Akhila datta dola
> > > 
> > > 共有していただきありがとうございます！
> > > 
> > > vLLMはbitsandbytesのようなオンザフライ4ビット量子化をサポートしていますか？一般的にgptqとawqと比較してどうですか？
> > > 
> > > 

---

> ## Luan Ngo Dinh
> 
> すばらしい共有をありがとうございます!!!
> 
> 推論プロセスで「128回中33回の推論エラー」が発生した理由を教えていただけますか？
> 
> > ## Chris Deotte（トピック作成者）
> > 
> > 128の入力テキストのうち33回は、モデルが生成テキストを予測できませんでした。したがって、これらの33回には確率を抽出するためのトークンがありません。私たちのtry/exceptコードブロックがこれをキャッチし、これらのケースでは[1/3, 1/3, 1/3]を予測します。
> > 
> > 
> > 
> > > ## Luan Ngo Dinh
> > > 
> > > モデルはファインチューニングされていても、トークン「a」、「b」、「tie」を生成しなかったという意味ですか？
> > > 
> > > 
> > > 
> > > ## Chris Deotte（トピック作成者）
> > > 
> > > はい。問題は、モデルをmax_model_len=1024に切り捨てたため（速度のため）、入力テキストが1024を超えるとvLLMが1024に切り捨てられ、生成されたテキストを出力するための余地がなくなるからです。よりスマートなプロンプトエンジニアリングまたは切り捨て戦略を使用することで、これらの推論エラーを回避できます。
> > > 
> > > 
> > > > ## floriandev
> > > > 
> > > > プロンプトエンジニアリングまたは切り捨て戦略に取り組んでいます。正しい方向に進んでいるかもしれないと聞いてうれしいです;-)
> > > > 

---

> ## Xinian Guo
> 
> こんにちは、このモデルはコンペティションデータでファインチューニングされていますか？
> 
> > ## Chris Deotte（トピック作成者）
> > 
> > はい。このモデル（ノートブックバージョン8）はコンペデータでファインチューニングされています。ノートブックバージョン6 [こちら](https://www.kaggle.com/code/cdeotte/infer-34b-with-vllm?scriptVersionId=188642633) はファインチューニングなしのゼロショットです。
> > 
> > 
> > 
> > > ## yechenzhi1
> > > 
> > > こんにちは、ファインチューニングされたバージョンのLBスコアは何ですか？
> > > 
> > > 
> > > > ## floriandev
> > > > 
> > > > 私はそのノートブックで0.972を得ました。頑張ってください;-)
> > > > 

---

> ## Qihang Wang
> 
> こんにちは、クリス、あなたのプロセスを確認したいです：
> 
> CMIIW
> 
> qloraファインチューニング4ビットモデル？
> 
> qloraをモデルにマージ？
> 
> 4ビットを変換？
> 
> AWQ量子化
> 
> vLLMを使用して推論
> 
> 私は最初の3つのステップにあまり慣れていないので、これがあなたのやり方かどうかは確信がありません。
> 
> もう少し詳しく説明していただけますか？